## For TPC-DS dataset with the following query tempalte:
SELECT ss_store_sk,ss_coupon_amt, AF(ss_sales_price)

FROM store_sales

WHERE ss_sold_date_sk BETWEEN low AND high


GROUP BY ss_store_sk,ss_quantity

In [1]:
from dbestclient.executor.executor import SqlExecutor

## Parameter Initialization

In [2]:
sqlExecutor = SqlExecutor()
sqlExecutor.execute("set csv_split_char='|'")
sqlExecutor.execute("set encoder='embedding'")
sqlExecutor.execute("set n_epoch=20")
sqlExecutor.execute("set n_mdn_layer_node_reg=50")          
sqlExecutor.execute("set n_mdn_layer_node_density=60")      
sqlExecutor.execute("set n_hidden_layer=1")                       
sqlExecutor.execute("set n_gaussians_reg=8")                
sqlExecutor.execute("set n_gaussians_density=12")            
sqlExecutor.execute("set n_embedding_dim=20")
sqlExecutor.execute(
    "set table_header="
    + "'ss_sold_date_sk|ss_sold_time_sk|ss_item_sk|ss_customer_sk|ss_cdemo_sk|ss_hdemo_sk|"
    + "ss_addr_sk|ss_store_sk|ss_promo_sk|ss_ticket_number|ss_quantity|ss_wholesale_cost|"
    + "ss_list_price|ss_sales_price|ss_ext_discount_amt|ss_ext_sales_price|"
    + "ss_ext_wholesale_cost|ss_ext_list_price|ss_ext_tax|ss_coupon_amt|ss_net_paid|"
    + "ss_net_paid_inc_tax|ss_net_profit|none'"
)

Local mode is on, as no slaves are provided.
start loading pre-existing models.
Loaded 4 models. time cost  0.267932 s
OK, csv_split_char is updated.
OK, encoder is updated.
OK, n_epoch is updated.
OK, n_mdn_layer_node_reg is updated.
OK, n_mdn_layer_node_density is updated.
OK, n_hidden_layer is updated.
OK, n_gaussians_reg is updated.
OK, n_gaussians_density is updated.
OK, n_embedding_dim is updated.
OK, table_header is updated.


## Model Creation

In [3]:
# sqlExecutor.execute("drop table template3")  # drop the model
sqlExecutor.execute(
    "create table template3(ss_sales_price real, ss_sold_date_sk real)  from '/home/quincy/Documents/workspace/data/tpcds/1t/ss_1m.csv' GROUP BY ss_store_sk,ss_quantity method uniform size 0.0003472"
)

Start creating model template3
The given table is treated as a uniform sample, and it is obtained with sampling rate 0.0003472
Reading data file...
get frequency info from data....
Starting training kde models for model template3
training regression...
********************************************************************************
finish training embedding.
embedding inference...
start normalizing data...
transform data from MDN training...
finish transforming data from MDN training...
< Epoch 0
< Epoch 1
< Epoch 2
< Epoch 3
< Epoch 4
< Epoch 5
< Epoch 6
< Epoch 7
< Epoch 8
< Epoch 9
< Epoch 10
< Epoch 11
< Epoch 12
< Epoch 13
< Epoch 14
< Epoch 15
< Epoch 16
< Epoch 17
< Epoch 18
< Epoch 19
Finish regression training.
training density...
********************************************************************************
finish training embedding.
< Epoch 0
< Epoch 1
< Epoch 2
< Epoch 3
< Epoch 4
< Epoch 5
< Epoch 6
< Epoch 7
< Epoch 8
< Epoch 9
< Epoch 10
< Epoch 11
< Epoch 12
< Epoch 1

## Query Serving

In [5]:
predictions = sqlExecutor.execute(
            "select ss_store_sk, sum(ss_sales_price)  from template3 where   2451119  <=ss_sold_date_sk<= 2451483  group by ss_store_sk,ss_quantity"
        )

# sqlExecutor.execute("drop table template3")

OK
       0            1
       , 1.186664e+08
      ,1 1.028813e+06
     ,10 1.115411e+06
    ,100 1.193234e+06
     ,11 1.226437e+06
     ,12 1.173658e+06
     ,13 1.267323e+06
     ,14 1.362054e+06
     ,15 1.099758e+06
     ,16 1.055497e+06
     ,17 1.291139e+06
     ,18 1.490364e+06
     ,19 8.588586e+05
      ,2 1.041302e+06
     ,20 1.111939e+06
     ,21 1.209664e+06
     ,22 1.082518e+06
     ,23 1.279682e+06
     ,24 1.313459e+06
     ,25 1.234011e+06
     ,26 1.276624e+06
     ,27 1.249453e+06
     ,28 1.241824e+06
     ,29 1.082484e+06
      ,3 1.269040e+06
     ,30 1.054618e+06
     ,31 1.183074e+06
     ,32 1.409190e+06
     ,33 1.246077e+06
     ,34 1.096488e+06
     ,35 1.423830e+06
     ,36 1.226637e+06
     ,37 1.219161e+06
     ,38 1.185323e+06
     ,39 1.036242e+06
      ,4 1.432005e+06
     ,40 1.353333e+06
     ,41 1.145399e+06
     ,42 1.332006e+06
     ,43 1.227648e+06
     ,44 1.241310e+06
     ,45 1.293181e+06
     ,46 1.092630e+06
     ,47 1.333068e+06
     ,4

## HIVE query
select ss_store_sk, count(ss_sales_price),sum(ss_sales_price),avg(ss_sales_price)  from store_sales_1t where  ss_sold_date_sk BETWEEN 2451119  AND 2451483  group by ss_store_sk,ss_quantity